# Merge the controlled values

In [228]:
import numpy as np
import pandas as pd

In [229]:
concatenated = pd.read_csv('../../data/concatenated.tsv', delimiter='\t', dtype=object)

### single column mapping

Writing a function for the executio of the single column mapping

In [233]:
def merge_control_vocabulary(concatenated, mapping, mappingname):
    """
    """
    merge_column = ''.join(['verbatim', ''.join([mappingname[0].upper(), mappingname[1:]])])
    merged = pd.merge(concatenated[["datasetName", merge_column]], mapping, how='left', 
         left_on=["datasetName", merge_column], right_on=['dataset', 'verbatimValue'])
    concatenated[mappingname] = merged["controlledValue"]
    return concatenated

List with the mappings that need to be done:

In [234]:
mappings = ['invasionStage', 'introductionPathway', 'habitat', 'nativeRange']

loop over the mapping and updating the concatenated file:

In [236]:
for term in mappings:
    print(term)
    mapping = pd.read_csv(''.join(["../../vocabularies/", term, "-mapping.tsv"]), delimiter='\t')
    concatenated = merge_control_vocabulary(concatenated, mapping, term)

invasionStage
introductionPathway
habitat
nativeRange


### multi-column mapping

In [238]:
term = 'presence'

In [239]:
mapping = pd.read_csv(''.join(["../../vocabularies/", term, "-mapping.tsv"]), delimiter='\t')

In [240]:
merge_columns = [term for term in mapping.columns if 'verbatim' in term]

In [241]:
merge_columns_mapping = ["dataset"] + merge_columns
merge_columns_concat = ["datasetName"] + merge_columns

In [243]:
merge_column = ''.join(['verbatim', ''.join([mappingname[0].upper(), mappingname[1:]])])
merged = pd.merge(concatenated[merge_columns_concat], mapping, how='left', 
     left_on=merge_columns_concat, right_on=merge_columns_mapping)


In [245]:
merge_columns_final = [''.join([term[8].lower(), term[9:]]) for term in merge_columns]
concatenated[merge_columns_final] = merged[merge_columns_final]

### Saving the concatenated

In [256]:
concatenated.to_csv('../../data/concatenated.tsv', delimiter='\t', dtype=object)

### checkup

Single column mappings:

In [285]:
for term in mappings:
    number_of_unmapped = sum(concatenated['verbatimNativeRange'].notnull() & concatenated['nativeRange'].isnull())
    if number_of_unmapped == 0:
        message = 'Great!'
    else:
        message = 'Bummer'        
    print(term, "has", str(number_of_unmapped), " badly mapped values. ", message)

invasionStage has 0  badly mapped values.  Great!
introductionPathway has 0  badly mapped values.  Great!
habitat has 0  badly mapped values.  Great!
nativeRange has 0  badly mapped values.  Great!


Multi column mapping

In [286]:
merge_columns_mapping

['dataset',
 'verbatimPresenceBE',
 'verbatimPresenceFL',
 'verbatimPresenceWA',
 'verbatimPresenceBR',
 'verbatimPresenceBECoast',
 'verbatimPresenceBPNS',
 'verbatimPresenceBEEZ']

In [291]:
sum(concatenated[merge_columns].notnull().all(axis=1) & concatenated[merge_columns_final].isnull().all(axis=1))

0